# Entering up to 150 lineups
FanDuel and DraftKings provide a daily CSV that allows players to upload multiple lineups. We bring the CSV to this notebook and then match the names in our APi with the names as they're written in the CSV.

In [239]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import requests
import sqlite3
import re

In [240]:
# #Which site are we playing?
site = input("Are you playing FanDuel or DraftKings?").lower()
if site == 'fanduel':
    site = 'FD'
else:
    site = 'DK'

Are you playing FanDuel or DraftKings? fanduel


In [241]:
today = (datetime.now()).strftime('%Y%m%d')

In [242]:
today

'20250221'

In [243]:
def clean_name(name):
    # Remove periods between initials like C.J., D.J. (case-sensitive)
    #name = re.sub(r'\b([A-Z])\.\s*([A-Z])\.\b', r'\1\2', name)
    
    # Remove common suffixes like Jr., Sr., III, II, IV (case-sensitive)
    cleaned_name = re.sub(r'(\,|\.|Sr|Jr|III|II|IV)', '', name).strip()
    
    return cleaned_name

In [244]:
#Finds names as they're written in FanDuel and DraftKings that aren't matched in API
def get_names_not_in_api(site_list, api_list):
    api_names_set = set(api_list)
    site_names_set = set(site_list)
    names_not_in_api = site_names_set.difference(api_names_set)
    return names_not_in_api

This is handy! When we trained the model we put together a dictionary for each site with the names paired with their matches in the api. We'll continually update them and then reverse the keys and values.

In [245]:
#Keys are the wrong way, values are names the way they are on the template
fd_names_to_change = {'Vince Williams Jr.': 'Vince Williams', 'Tim Hardaway Jr.': 'Tim Hardaway', 'Wendell Carter Jr.': 'Wendell Carter'}


# = {'Nic Claxton': 'Nicolas Claxton', 'Moritz Wagner': 'Moe Wagner', 'Tristan da Silva': 'Tristan Da Silva',\
#                      'Dominick Barlow': 'Dom Barlow', 'GG Jackson': 'Gregory Jackson', 'KJ Martin': 'Kenyon Martin',\
#                       'Cam Thomas': 'Cameron Thomas', 'Ish Wainright': 'Ishmail Wainright', 'Jeenathan Williams': 'Nate Williams'}

In [246]:
dk_names_to_change = {'Nic Claxton': 'Nicolas Claxton', 'Moritz Wagner': 'Moe Wagner', 'Alexander Sarr': 'Alex Sarr',\
                     'Dominick Barlow': 'Dom Barlow', 'KJ Martin': 'Kenyon Martin', 'Cam Thomas': 'Cameron Thomas',\
                     'Ronald Holland': 'Ron Holland', 'Marjon Beauchamp': 'MarJon Beauchamp', 'David Jones Garcia': 'David Jones',\
                     'Jeenathan Williams': 'Nate Williams', 'Ish Wainright': 'Ishmail Wainright', 'Guillermo Hernangomez': 'Willy Hernangomez'}

In [247]:
name_changes_fd = {v: k for k, v in fd_names_to_change.items()}
name_changes_dk = {v: k for k, v in dk_names_to_change.items()}

In [248]:
name_changes_fd

{'Vince Williams': 'Vince Williams Jr.',
 'Tim Hardaway': 'Tim Hardaway Jr.',
 'Wendell Carter': 'Wendell Carter Jr.'}

In [249]:
#This is the CSV downloaded from the site
site_df = pd.read_csv(f"{site}_Template_{today}.csv")

In [250]:
#We process the CSV in Google Sheets so that we have a column for each position
#and a column with each player's name as it's written on the site
#We also need the Player ID + Player Name column from the template
site_df

,PG,PG.1,SG,SG.1,SF,SF.1,PF,PF.1,C,Unnamed: 9,Player ID + Player Name,Nickname
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-40199:Giannis Antetokounmpo,Giannis Antetokounmpo
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-84680:Shai Gilgeous-Alexander,Shai Gilgeous-Alexander
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-157808:Cade Cunningham,Cade Cunningham
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-58462:Karl-Anthony Towns,Karl-Anthony Towns
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-145321:Anthony Edwards,Anthony Edwards
...,...,...,...,...,...,...,...,...,...,...,...,...
310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-203763:Nikola Topic,Nikola Topic
311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-203760:Pacome Dadiet,Pacome Dadiet
312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-188411:Taylor Hendricks,Taylor Hendricks
313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-188414:Victor Wembanyama,Victor Wembanyama


In [251]:
##Bring in the CSV with all the lineups
lineups_df = pd.read_csv(f"{site}_lineups_{today}.csv")

In [252]:
lineups_df.head()

,PG,SG,SF,PF,C,player_id,Name,Team,Salary,FD_Pred,FD_Value,lineup_id,LU_Pos
0,False,False,False,True,True,28118309129,Domantas Sabonis,SAC,9800,40.213223,4.103390,149,PF
1,False,False,False,False,True,28398922762,Bismack Biyombo,SA,4300,24.730015,5.751166,149,C
2,False,False,True,True,False,28788530069,Lauri Markkanen,UTA,6700,27.820807,4.152359,149,SF
3,False,True,True,False,False,28958935349,Tim Hardaway Jr.,DET,4000,18.939606,4.734901,149,SG
4,False,True,True,False,False,94344202027,Anthony Edwards,MIN,9900,40.123684,4.052897,149,SG


In [253]:
#The names as they're written on the site template
template_names = list(site_df['Nickname'])

In [254]:
#The names as they're written in the linups
lineup_names = list(lineups_df['Name'].unique())

In [255]:
#We should create a dict with each names and num of lineups for each player, but for now this is a list of all
#the names in any lineup
lineup_names

['Domantas Sabonis',
 'Bismack Biyombo',
 'Lauri Markkanen',
 'Tim Hardaway Jr.',
 'Anthony Edwards',
 'Keyonte George',
 'Jalen Williams',
 'Stephon Castle',
 'Jonathan Mogbo',
 'Giannis Antetokounmpo',
 'Max Christie',
 'Evan Mobley',
 'Kyshawn George',
 'Jaden McDaniels',
 'Khris Middleton',
 'DeMar DeRozan',
 'Wendell Carter Jr.',
 'Jordan Hawkins',
 'Nikola Jovic',
 'Desmond Bane',
 'Zach LaVine',
 'Cole Anthony',
 'Karl-Anthony Towns',
 'Cade Cunningham',
 'Jabari Smith',
 'Duncan Robinson',
 'Vince Williams Jr.',
 'Tyler Herro',
 'John Collins',
 'Mikal Bridges',
 'Josh Hart',
 'Luke Kennard',
 'Amen Thompson',
 'RJ Barrett',
 'Marcus Smart',
 'Bilal Coulibaly',
 'Isaiah Collier',
 'Chris Boucher',
 'Malik Monk',
 'Draymond Green',
 'CJ McCollum',
 'Malik Beasley',
 'Terry Rozier']

In [256]:
len(template_names), len(lineup_names)

(315, 43)

In [257]:
template_set = set(template_names)

In [258]:
lineup_set = set(lineup_names)

In [259]:
#Get names in lineups that aren't on template
lineup_names_not_on_template= lineup_set.difference(template_set)

In [260]:
lineup_names_not_on_template

{'Tim Hardaway Jr.', 'Vince Williams Jr.', 'Wendell Carter Jr.'}

In [261]:
template_list = list(template_set)

In [262]:
lineup_not_on_template = list(lineup_names_not_on_template)

# Fuzzy matching
Using fuzzy matching to find a names on the template that could match the unmatched names in the lineups.

In [276]:
from rapidfuzz import process, fuzz

In [277]:
SIMILARITY_THRESHOLD = 70

In [278]:
#This function takes the unmatched lineup name and looks for matchins in the template
def fuzzy_match(name, dk_names):
    match, score, _ = process.extractOne(name, dk_names, scorer=fuzz.token_sort_ratio)
    return match if score >= SIMILARITY_THRESHOLD else None

In [281]:
matches_on_template = [fuzzy_match(x, template_list) for x in lineup_not_on_template]

In [282]:
matches_on_template

['Vince Williams', 'Tim Hardaway', 'Wendell Carter']

In [283]:
#Making a dictionary
names_to_change = dict(zip(lineup_not_on_template, matches_on_template))

In [284]:
names_to_change

{'Vince Williams Jr.': 'Vince Williams',
 'Tim Hardaway Jr.': 'Tim Hardaway',
 'Wendell Carter Jr.': 'Wendell Carter'}

In [285]:
lineups_df['Name'] = lineups_df['Name'].replace(names_to_change)

In [286]:
#So now that all the names match ...

In [287]:
lineups_df.head(20)

,PG,SG,SF,PF,C,player_id,Name,Team,Salary,FD_Pred,FD_Value,lineup_id,LU_Pos
0,False,False,False,True,True,28118309129,Domantas Sabonis,SAC,9800,40.213223,4.103390,149,PF
1,False,False,False,False,True,28398922762,Bismack Biyombo,SA,4300,24.730015,5.751166,149,C
2,False,False,True,True,False,28788530069,Lauri Markkanen,UTA,6700,27.820807,4.152359,149,SF
3,False,True,True,False,False,28958935349,Tim Hardaway,DET,4000,18.939606,4.734901,149,SG
4,False,True,True,False,False,94344202027,Anthony Edwards,MIN,9900,40.123684,4.052897,149,SG
5,True,True,False,False,False,945447821869,Keyonte George,UTA,6200,25.947900,4.185145,149,PG
6,False,False,True,True,False,94624262027,Jalen Williams,OKC,8300,33.859955,4.079513,149,SF
7,True,True,False,False,False,948940097989,Stephon Castle,SA,5800,24.131907,4.160674,149,PG
8,False,False,False,True,True,949842011869,Jonathan Mogbo,TOR,4900,23.134827,4.721393,149,PF
9,False,False,False,True,False,28118035349,Giannis Antetokounmpo,MIL,11900,48.221046,4.052189,148,PF


In [288]:
site_df.head()

,PG,PG.1,SG,SG.1,SF,SF.1,PF,PF.1,C,Unnamed: 9,Player ID + Player Name,Nickname
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-40199:Giannis Antetokounmpo,Giannis Antetokounmpo
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-84680:Shai Gilgeous-Alexander,Shai Gilgeous-Alexander
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-157808:Cade Cunningham,Cade Cunningham
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-58462:Karl-Anthony Towns,Karl-Anthony Towns
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-145321:Anthony Edwards,Anthony Edwards


In [289]:
site_df = site_df.rename(columns = {'Nickname':'Name'})

In [290]:
#Creating a merge_df to add names and ids to lineups_df
merge_df = site_df[['Player ID + Player Name', 'Name']]

In [291]:
lineups_df = pd.merge(lineups_df, merge_df, on = 'Name', how = 'left')

In [292]:
lineups_df.head(20)

,PG,SG,SF,PF,C,player_id,Name,Team,Salary,FD_Pred,FD_Value,lineup_id,LU_Pos,Player ID + Player Name
0,False,False,False,True,True,28118309129,Domantas Sabonis,SAC,9800,40.213223,4.103390,149,PF,113346-59358:Domantas Sabonis
1,False,False,False,False,True,28398922762,Bismack Biyombo,SA,4300,24.730015,5.751166,149,C,113346-14505:Bismack Biyombo
2,False,False,True,True,False,28788530069,Lauri Markkanen,UTA,6700,27.820807,4.152359,149,SF,113346-80812:Lauri Markkanen
3,False,True,True,False,False,28958935349,Tim Hardaway,DET,4000,18.939606,4.734901,149,SG,113346-15846:Tim Hardaway
4,False,True,True,False,False,94344202027,Anthony Edwards,MIN,9900,40.123684,4.052897,149,SG,113346-145321:Anthony Edwards
5,True,True,False,False,False,945447821869,Keyonte George,UTA,6200,25.947900,4.185145,149,PG,113346-188412:Keyonte George
6,False,False,True,True,False,94624262027,Jalen Williams,OKC,8300,33.859955,4.079513,149,SF,113346-171760:Jalen Williams
7,True,True,False,False,False,948940097989,Stephon Castle,SA,5800,24.131907,4.160674,149,PG,113346-203723:Stephon Castle
8,False,False,False,True,True,949842011869,Jonathan Mogbo,TOR,4900,23.134827,4.721393,149,PF,113346-203799:Jonathan Mogbo
9,False,False,False,True,False,28118035349,Giannis Antetokounmpo,MIL,11900,48.221046,4.052189,148,PF,113346-40199:Giannis Antetokounmpo


In [294]:
# Define the desired sort order for FanDuel
#Eventually we'll need to do this for DraftKings, too
if site == 'FD':
    sort_order = {
        "PG": 1, "PG": 2, "SG": 3, "SG": 4, "SF": 5, "SF": 6, "PF": 7, "PF": 8, "C": 9}
else:
    sort_order = {
        "PG": 1, "SG": 2, "G": 3, "SF": 4, "PF": 5, "F": 6, "C": 7, "UTIL": 8}

In [295]:
# Define the shape of the blank DataFrame
num_rows = len(site_df)
num_cols = 9 if 'UTIL' not in lineups_df.columns else 8
#Getting a list of the columns except for Nickname
columns = list(site_df.columns)[:-1]
#columns = [f'col_{i}' for i in range(num_cols)]  # Example column names

# Create an empty DataFrame
#df = pd.DataFrame(index=range(num_rows), columns=columns)

# Filling values row by row, iterating over columns
for row in range(num_rows):
    temp_df = lineups_df[lineups_df['lineup_id'] == row]
    sorted_df = temp_df.copy()
    sorted_df["Sort_Order"] = sorted_df["LU_Pos"].map(sort_order)
    temp_df = sorted_df.sort_values(by="Sort_Order").drop(columns=["Sort_Order"]).reset_index(drop=True)
    names_to_fill = list(temp_df['Player ID + Player Name'])
    # Iterate through columns and fill with names from the list
    for i, col in enumerate(columns):
        if i < len(names_to_fill):  # Ensure we don't go out of bounds
            site_df.at[row, col] = names_to_fill[i]  # Fill column with corresponding name

### Then we save site_df as a CSV and feed it to FanDuel


C:\Users\Owner\AppData\Local\Temp\ipykernel_26916\1052267123.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '113346-19067:CJ McCollum' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  site_df.at[row, col] = names_to_fill[i]  # Fill column with corresponding name
C:\Users\Owner\AppData\Local\Temp\ipykernel_26916\1052267123.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '113346-157808:Cade Cunningham' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  site_df.at[row, col] = names_to_fill[i]  # Fill column with corresponding name
C:\Users\Owner\AppData\Local\Temp\ipykernel_26916\1052267123.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '113346-84670

In [296]:
site_df

,PG,PG.1,SG,SG.1,SF,SF.1,PF,PF.1,C,Unnamed: 9,Player ID + Player Name,Name
0,113346-19067:CJ McCollum,113346-157808:Cade Cunningham,113346-84670:Mikal Bridges,113346-171756:Vince Williams,113346-188416:Bilal Coulibaly,113346-110357:RJ Barrett,113346-68203:Chris Boucher,113346-66236:John Collins,113346-15860:Draymond Green,NaN,113346-40199:Giannis Antetokounmpo,Giannis Antetokounmpo
1,113346-19067:CJ McCollum,113346-203726:Isaiah Collier,113346-84670:Mikal Bridges,113346-40783:Zach LaVine,113346-40562:Josh Hart,113346-188416:Bilal Coulibaly,113346-68203:Chris Boucher,113346-110357:RJ Barrett,113346-15860:Draymond Green,NaN,113346-84680:Shai Gilgeous-Alexander,Shai Gilgeous-Alexander
2,113346-19067:CJ McCollum,113346-203726:Isaiah Collier,113346-84670:Mikal Bridges,113346-188416:Bilal Coulibaly,113346-9714:DeMar DeRozan,113346-188405:Amen Thompson,113346-68203:Chris Boucher,113346-66236:John Collins,113346-15860:Draymond Green,NaN,113346-157808:Cade Cunningham,Cade Cunningham
3,113346-19067:CJ McCollum,113346-157808:Cade Cunningham,113346-84670:Mikal Bridges,113346-15846:Tim Hardaway,113346-188416:Bilal Coulibaly,113346-110357:RJ Barrett,113346-68203:Chris Boucher,113346-66236:John Collins,113346-15860:Draymond Green,NaN,113346-58462:Karl-Anthony Towns,Karl-Anthony Towns
4,113346-19067:CJ McCollum,113346-203726:Isaiah Collier,113346-84670:Mikal Bridges,113346-188416:Bilal Coulibaly,113346-80812:Lauri Markkanen,113346-188405:Amen Thompson,113346-68203:Chris Boucher,113346-110357:RJ Barrett,113346-15860:Draymond Green,NaN,113346-145321:Anthony Edwards,Anthony Edwards
...,...,...,...,...,...,...,...,...,...,...,...,...
310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-203763:Nikola Topic,Nikola Topic
311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-203760:Pacome Dadiet,Pacome Dadiet
312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-188411:Taylor Hendricks,Taylor Hendricks
313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113346-188414:Victor Wembanyama,Victor Wembanyama


In [297]:
#Filtering out non-position columns
site_df = site_df[['PG', 'PG.1', 'SG', 'SG.1', 'SF', 'SF.1', 'PF', 'PF.1', 'C']]

In [301]:
#Taking the .1 out of the column names
#This might not apply to SK, we'll have to see
site_df.columns = site_df.columns.str.replace(r'\.\d+', '', regex=True)

In [299]:
site_df

,PG,PG,SG,SG,SF,SF,PF,PF,C
0,113346-19067:CJ McCollum,113346-157808:Cade Cunningham,113346-84670:Mikal Bridges,113346-171756:Vince Williams,113346-188416:Bilal Coulibaly,113346-110357:RJ Barrett,113346-68203:Chris Boucher,113346-66236:John Collins,113346-15860:Draymond Green
1,113346-19067:CJ McCollum,113346-203726:Isaiah Collier,113346-84670:Mikal Bridges,113346-40783:Zach LaVine,113346-40562:Josh Hart,113346-188416:Bilal Coulibaly,113346-68203:Chris Boucher,113346-110357:RJ Barrett,113346-15860:Draymond Green
2,113346-19067:CJ McCollum,113346-203726:Isaiah Collier,113346-84670:Mikal Bridges,113346-188416:Bilal Coulibaly,113346-9714:DeMar DeRozan,113346-188405:Amen Thompson,113346-68203:Chris Boucher,113346-66236:John Collins,113346-15860:Draymond Green
3,113346-19067:CJ McCollum,113346-157808:Cade Cunningham,113346-84670:Mikal Bridges,113346-15846:Tim Hardaway,113346-188416:Bilal Coulibaly,113346-110357:RJ Barrett,113346-68203:Chris Boucher,113346-66236:John Collins,113346-15860:Draymond Green
4,113346-19067:CJ McCollum,113346-203726:Isaiah Collier,113346-84670:Mikal Bridges,113346-188416:Bilal Coulibaly,113346-80812:Lauri Markkanen,113346-188405:Amen Thompson,113346-68203:Chris Boucher,113346-110357:RJ Barrett,113346-15860:Draymond Green
...,...,...,...,...,...,...,...,...,...
310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [300]:
#Saving the file that will be uploaded to the site
site_df.to_csv(f"{site}_upload_{today}.csv", index = False, encoding='utf-8')